# Automation Workflow

This notebook is in its early stages and its purpose is for automating a water pixel count process for Sentinel-2 imagery across multiple dates. This work will be expanded upon in the coming months. It closes follows the structure for the Stream Gage vs. Sentinel-2 Workflow.

In [1]:
!pip install geemap

In [2]:
import hydrofunctions as hf
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import folium
import geemap.eefolium as geemap
import ee

%matplotlib inline

In [3]:
ee.Authenticate()
ee.Initialize()  

Enter verification code: 4/1AX4XfWiBHMWiaEYB21eiJDTdsd4eTbpPUwGEwM5mWSfjGQVRgy4qzNAu8BU

Successfully saved authorization token.


In [4]:
Map = geemap.Map(center=[32.3,-90.9], zoom = 11)
Map

In [41]:
roi = ee.Geometry.Point([-90.9,32.3])
#roi = ee.Geometry.Point([-114.762293, 36.06462])
#roi = ee.Geometry.Point([40.0150,105.2705])



In [42]:
#set start date, end date, collect images from'COPERNICUS/S2_SR'

start_date = '2019-03-16'
end_date = '2019-05-17'
#cloud_threshold = 0.1
ndwi_threshold = 0

images = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterBounds(roi)
    .filterDate(start_date, end_date)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)
    
))

print(images.size().getInfo())
print(images.aggregate_array('system:id').getInfo())

5
['COPERNICUS/S2_SR/20190322T163951_20190322T164744_T15SXR', 'COPERNICUS/S2_SR/20190327T164109_20190327T164602_T15SXR', 'COPERNICUS/S2_SR/20190421T163901_20190421T164917_T15SXR', 'COPERNICUS/S2_SR/20190426T163849_20190426T165038_T15SXR', 'COPERNICUS/S2_SR/20190516T163849_20190516T165438_T15SXR']


In [43]:
dates = images.aggregate_array('system:time_start').map(
    lambda d: ee.Date(d).format('YYYY-MM-dd')
)
print(dates.getInfo())

['2019-03-22', '2019-03-27', '2019-04-21', '2019-04-26', '2019-05-16']


In [67]:
def extract_water(img):

    ndwi_image = img.normalizedDifference(['B3', 'B8'])
    water_image = ndwi_image.gt(ndwi_threshold)
    return water_image


In [68]:
water_pix_images = images.map(extract_water)

In [81]:
first_water_pix_image = ee.Image(ndwi_images.toList(ndwi_images.size()).get(0)).selfMask()
#print(first_ndwi_image)

In [82]:
newMap = geemap.Map(center=[32.3,-90.9], zoom = 11)
newMap.addLayer(first_water_pix_image, {'palette': 'blue'}, 'First NDWI')
newMap.addLayerControl()
newMap

# count water pixels 

so for a date in date list (associated with imagecollection):
- extract water pixels for each date


In [78]:
count = []
for image in water_pix_images:
    
     image.reduceRegion(**{
    'reducer': ee.Reducer.count(),
    'geometry': image.geometry(),
    'scale': 10,
    'maxPixels': 1e9
})

TypeError: 'ImageCollection' object is not iterable